0.1 Import packages

In [2]:
import requests
import pandas as pd

0.2 Some useful url's for the MediaDive api

In [4]:
#MediaDive urls
comp_url = f'https://mediadive.dsmz.de/rest/medium/' #comp can be a mix of solutions and ingredients
taxa_url = f'https://mediadive.dsmz.de/rest/medium-strains/'
solu_url = f'https://mediadive.dsmz.de/rest/solution/' #returns ingredients
ingr_url = f'https://mediadive.dsmz.de/rest/ingredient/' #simplest form (I think)
    # All followed by a DSMZ media ID 

1.1 Function to list all components (less useful)

In [6]:
id_list = ['1', '1001'] #List of DSMZ media IDs

def get_composition_list():
    base_url = 'https://mediadive.dsmz.de/rest/medium/{}'
    composition_data = []

    for id in id_list:
        url = base_url.format(id)
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            # Traverse the nested structure to extract compounds
            solutions = data.get('data', {}).get('solutions', [])
            for solution in solutions:
                recipe = solution.get('recipe', [])
                for item in recipe:
                    if 'compound' in item:  # Check for 'compound' and 'compound_id'
                        composition_data.append({
                            'component': item.get('compound'),
                            'component_id': item.get('compound_id')
                        })
                    elif 'solution' in item:  # Check for 'solution' and 'solution_id'
                        composition_data.append({
                            'component': item.get('solution'),
                            'component_id': item.get('solution_id')
                        })
        else:
            print(f"Request failed with status code: {response.status_code}")

    # Convert the list of dictionaries to a DataFrame
    composition_df = pd.DataFrame(composition_data)
    return composition_df

In [7]:
#comp_list = get_composition_list()
#comp_list

1.2 Function to list a 'media id', its 'components', and the 'component ids'

In [9]:
id_list = ['1', '1001'] #List of DSMZ media IDs

def get_composition():
    base_url = 'https://mediadive.dsmz.de/rest/medium/{}'
    composition_data = []

    for id in id_list:
        url = base_url.format(id)
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            # Initialize lists to store components and ids for each media
            components = []
            component_ids = []
            # Traverse the nested structure to extract compounds
            solutions = data.get('data', {}).get('solutions', [])
            for solution in solutions:
                recipe = solution.get('recipe', [])
                for item in recipe:
                    if 'compound' in item:  # Check for 'compound' and 'compound_id'
                        components.append(item.get('compound'))
                        component_ids.append(item.get('compound_id'))
                    elif 'solution' in item:  # Check for 'solution' and 'solution_id'
                        components.append(item.get('solution'))
                        component_ids.append(item.get('solution_id'))
            # Append data for this media to composition_data
            composition_data.append({
                'media_id': id,
                'components': components,
                'component_ids': component_ids
            })     
        else:
            print(f"Request failed with status code: {response.status_code}")
    
        print(f'Retrieved data for {id}')

    # Convert the list of dictionaries to a DataFrame
    composition_df = pd.DataFrame(composition_data)
    return composition_df

# Can add something to get the compounds and IDs for the complex solution mixtures, but maybe easier to leave as "Vitamin mixture" etc for now
    # i.e., in the place of compounds some api entries were solutions, but each of those solutions has a list of compounds

In [10]:
media_comps = get_composition()
media_comps

Retrieved data for 1
Retrieved data for 1001


,media_id,components,component_ids
0,1,"[Peptone, Meat extract, Agar, Distilled water]","[1, 2, 3, 4]"
1,1001,"[NaHCO3, NH4Cl, NaH2PO4 x H2O, KCl, Vitamin mi...","[55, 42, 282, 61, 2043, 2044, 4, 731, 4, 55, 1..."


1.3 Function to list the 'strain_id', 'species', 'ccno' (culture collection number), and 'bacdive_id' for associated organisms of a given media

In [12]:
import requests
import pandas as pd

id_list = ['1','1001']  # List of DSMZ media IDs

def get_strains():
    base_url = 'https://mediadive.dsmz.de/rest/medium-strains/{}'
    strain_data = []

    for id in id_list:
        url = base_url.format(id)
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            
            strains = data.get('data', [])
            for strain in strains:
                strain_data.append({
                    'media_id': id,
                    'strain_id': strain.get('id'),
                    'species': strain.get('species'),
                    'ccno': strain.get('ccno'),
                    'bacdive_id': strain.get('bacdive_id')
                })     
        else:
            print(f"Request failed with status code: {response.status_code}")
    
        print(f'Retrieved data for {id}')

    # Convert the list of dictionaries to a DataFrame
    strain_df = pd.DataFrame(strain_data)
    return strain_df

In [13]:
strains_df = get_strains()
strains_df

Retrieved data for 1
Retrieved data for 1001


,media_id,strain_id,species,ccno,bacdive_id
0,1,1,Heyndrickxia coagulans,DSM 1,654.0
1,1,2,Paenibacillus macquariensis subsp. macquariensis,DSM 2,11477.0
2,1,3,Sporosarcina psychrophila,DSM 3,11984.0
3,1,6,Peribacillus psychrosaccharolyticus,DSM 6,748.0
4,1,7,Bacillus amyloliquefaciens,DSM 7,598.0
...,...,...,...,...,...
2291,1,49035,Acinetobacter pittii,DSM 101622,NaN
2292,1,49036,Acinetobacter pittii,DSM 101623,NaN
2293,1,1431,Pseudomonas sp.,DSM 1991,12943.0
2294,1,180,Pseudomonas putida,DSM 291,12871.0


In [29]:
# Maybe filter for type strains?